In [ ]:
import matplotlib.pyplot as plt
import uproot as urt
import numpy as np
import mplhep as hep
import sys
# adding Folder_2 to the system path
sys.path.insert(0, '../lib')
import json
from treeIO import varData, ROOTDataset, getFile, rangeData
from plotHelper import DrawVarHist

In [ ]:
nFileMCPR = getFile('MC','Prompt')[0]
MCLoader = ROOTDataset(nFileMCPR, "mvaTupler/muonTuple", varData, '60 MB')

nFileMCNR = getFile('MC','NonPrompt')[0]
MCLoader = ROOTDataset(nFileMCNR, "mvaTupler/muonTuple", varData, '60 MB')

In [ ]:
varArr = {d : np.empty(0) for d in varData}
uniformHistbin =[1,3,5,7,12,20,100] 
histArr = {
    "DATA" : {
        d : np.histogram([], bins=uniformHistbin)for d in varData
    },
    "MCPR" : {
        i : {d : np.histogram([], bins=uniformHistbin)for d in varData } for i in [-999] + [ x for x in range(-11,12) ]
    }
}
hist2DArr = {
    "DATA" :{
    },
    "MCPR" :{
        'matching_' : ''
    }
}


In [ ]:
print(rangeData['local_chi2'][1]) 

In [ ]:

for i, data in enumerate(MCLoader):
    print(i)
    for tup in data[1]:
        # print(data[1])
        keyList = list(data[1].keys())
        dataValues = np.array(list(data[1].values())).T
        # print(dataValues.shape)
        for matchIdx in [-999] + [ x for x in range(-11,12) ]:
            if i == 0 :
                histArr["MCPR"][matchIdx][tup] = np.histogram(
                    dataValues[dataValues[:,keyList.index('matching')] == matchIdx][:,keyList.index(tup)],
                    bins=int(rangeData[tup][1]), 
                    range=tuple(rangeData[tup][0])
                    )
                # print(tup+' ',histArr["MCPR"][matchIdx][tup][1])
            else :
                histArr["MCPR"][matchIdx][tup][0][:] = np.histogram(
                    dataValues[dataValues[:,keyList.index('matching')] == matchIdx][:,keyList.index(tup)],
                    histArr["MCPR"][matchIdx][tup][1]
                    )[0][:]
                # print(tup+' ',histArr["MCPR"][matchIdx][tup][1])
    if i > 6:
        break

In [ ]:
densityArr = {
    "DATA" : { 
    },
    "MCPR": {
        # d : np.histogram( histArr["MCPR"][d][1][1:], histArr["MCPR"][d][1], weights=(histArr["MCPR"][d][0]/histArr["MCPR"][d][0].sum()), density=True ) for d in varData 
        i : {d : np.histogram( histArr["MCPR"][i][d][1][1:], histArr["MCPR"][i][d][1], weights=(histArr["MCPR"][i][d][0]/histArr["MCPR"][i][d][0].sum()), density=True ) for d in varData } for i in [-999] + [ x for x in range(-11,12) ]
    }
}

In [ ]:
hep.style.use("CMS");

for var in varData :
    fig, axs = plt.subplot_mosaic([['plt','plt','plt', 'leg']], figsize=(15,11), empty_sentinel='leg');
    hep.cms.label('Internal', data=True, lumi=1.71, year=2023, com=5.36);
    hep.histplot(
        [ _p_ for _p_ in [[densityArr['MCPR'][i][var] for i in [-999] + [ x for x in range(-11,12)]]][0]],
        ax=axs['plt'],
        histtype='step',
        label=list(densityArr["MCPR"].keys()),
        # label=[ i for i in [-999] + [ x for x in range(-11,12) ]]
        )    
    axs['plt'].legend(loc='upper right', bbox_to_anchor=(1.4, 1), borderaxespad=0., fontsize='xx-small', ncols=2)
    plt.text(0.89, 0.7, var, horizontalalignment='right', transform=axs['plt'].transAxes)
    plt.savefig('variables/MCSimType/'+var+'.png')
    # break;